In [1]:
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import numpy as np
import os.path
import csv
from sklearn.svm import SVC

class PatientPhenotype:
    
    def __init__(self, eid, case, sex, yearBirth):
        
        self.eid = eid
        self.case = case
        self.sex = sex
        self.yearBirth = yearBirth
        self.snps = {}
        
    def getEid(self):
        return self.eid
     
    def getCase(self):
        return self.case
    
    def getSex(self):
        return self.sex
    
    def getYearBirth(self):
        return self.yearBirth
        
    def addSnps(self, snpId, allele1,allele2):
        self.snps[snpId] = Snp(snpId,allele1,allele2)
        
    def snpCode(self,snpId,allele1,allele2):
       
        self.snps[snpId].setSnpCode(allele1,allele2)
        
    def getSnpCode(self,snpId):
        return self.snps[snpId].getSnpCode()
        
    def getSize(self):
        return len(self.snps)
        
        
class Snp:
    
    def __init__(self,snpId,allele1,allele2):
        
        self.snpId = snpId
        self.allele1 = allele1
        self.allele2 = allele2
        self.snpCode = -1
        
    def getId(self):
        
        return self.snpId
        
    def getAllele1(self):
        
        return self.allele1
        
    def getAllele2(self):
        
        return self.allele2
        
    def setSnpCode(self,allele1,allele2):
      
        if self.allele1 == allele1 and self.allele2 == allele1:
            code = 2
           
        elif self.allele1 == allele1 and self.allele2 != allele1:
            code = 1
           
        elif self.allele1 != allele1 and self.allele2 == allele1:
            code = 1
            
        elif self.allele1 != allele1 and self.allele2 != allele1:
            code = 0
            
        self.snpCode = code
        
    def getSnpCode(self):
        
        return self.snpCode
    
    
class SaveDataSet():
    def __init__(self,name,ids,data):
        
        self.name = name
        self.ids = ids
        self.data = data
        self.__writeLogFile()
        self.__writeVariables('patients')
        self.__writeVariables('snps')
        self.__writeCSV()
        
    def __writeLogFile(self):
        
        write = open(self.name+'.log','w')
        write.write("Log file for the Test Data Set "+self.name+'\n')
        write.write(self.name + " has "+ str(len(self.ids['patients']['nameToId'].keys())) + " patients "'\n')
        write.write(self.name + " has "+ str(len(self.ids['snps']['nameToId'].keys())) + " snps "'\n')
        write.close()
        
    def __writeVariables(self,kind):
        
        write = open(self.name+'_'+kind+'.txt','w')
        write.write(kind + '\n')
        
        for i in range(len(self.ids[kind]['nameToId'].keys())):
            name = self.ids[kind]['idToName'][i]
            write.write(name + '\t'+str(self.ids[kind]['nameToId'][name]) + '\n')
        
        write.close()
        
        
    def __writeCSV(self):
        
        try:
            
            with open(self.name+'.csv', 'w') as csvfile:
                writer = csv.writer(csvfile, dialect='excel', quoting=csv.QUOTE_NONNUMERIC)
            
                line = []
                for i in range(len(self.ids['snps']['nameToId'].keys())):
                   # if i == 0 :
                    #    line += self.ids['snps']['idToName'][i]
                    #else:
                    line.append(self.ids['snps']['idToName'][i])
                writer.writerow(line)
            
                line = [] 
                for i in range(len(self.ids['snps']['nameToId'].keys())):
                    name = self.ids['snps']['idToName'][i]
                 #   if i == 0:
                  #      line += str(self.ids['snps']['nameToId'][name])
                   # else:
                    line.append(str(self.ids['snps']['nameToId'][name]))
                writer.writerow(line)
            
            
                for i in range(len(self.ids['patients']['nameToId'].keys())):
                    line = []
                    for j in range(len(self.ids['snps']['nameToId'].keys())):
                        #if j == 0:
                        #    line += str(self.data[i,j]) 
                        #else:
                        
                        line.append(str(self.data[i,j]))
                        
                    writer.writerow(line)
        finally:
            pass   
        

def setIdToName(aList):
    
    ids = {}
    nameToId = {}
    idToName = {}
    count = 0
    
    for i in aList:
        
        nameToId[i] = count
        idToName[count] = i
        count += 1
        
    ids['nameToId'] = nameToId
    ids['idToName'] = idToName
    
    return ids


def createDataSet(kind,chromos,patients):
    
    for i in range(len(chromos.keys())):
    
        chro = 'chr'+str(i+1)
        path = 'C:\\Users\\ANTONIS\\Desktop\\test1\\'+chro+kind+'.lgen'
    
        if os.path.exists(path):
            f = open(path,'r')
    
            for line in f:
                patients[line.split()[0]].addSnps(line.split()[2], line.split()[3],line.split()[4])
    

    
    for i in range(chromos.keys()):
    
        chro = 'chr'+str(i+1)
    
        for snp in chromos[chro].keys():
        
            allele1 = chromos[chro][snp][0]
            allele2 = chromos[chro][snp][1]
    
            for patient in patients.keys():
            
                patients[patient].snpCode(snp,allele1,allele2)
    
    return patients

In [2]:
minorSnps = []
chromosomes = {}

for i in range(14):
    
    chro = 'chr'+str(i+1)
    path = 'C:\\Users\\ANTONIS\\Desktop\\test1\\'+chro+'.assoc.fisher'
    f = open(path,'r')
    f.readline()
    snps = {}
    
    for line in f:
        alleles = []
        minorSnps.append(line.split()[1])
        alleles.append(line.split()[3])
        alleles.append(line.split()[6])
        snps[line.split()[1]] = alleles
        
    chromosomes[chro] = snps
    
for i in range(14):
    
    chro = 'chr'+str(i+1)
    path = 'C:\\Users\\ANTONIS\\Desktop\\test1\\'+chro+'snpList.txt'
    write = open(path,'w')
    for snp in chromosomes[chro].keys():
        write.write(snp + '\n')
            
    write.close()
    


# TRAIN SET CREATION

In [3]:
########   TRAIN ##########

patientsPhen = {}

f = open('C:\\Users\\Antonis\\Documents\\GitHub\\diplwmatikh\\data\\phenotype_euro_train.txt','r')
f.readline()

for line in f:
    patientsPhen[line.split()[0]] = PatientPhenotype(line.split()[0],line.split()[3],line.split()[1],line.split()[2])

    
path = 'C:\\Users\\ANTONIS\\Desktop\\test1\\'+'trainPatient.txt'
write = open(path,'w')
for patient in patientsPhen.keys():
    write.write(patient + '\n')
            
write.close()
    





# run train_lgen bat

In [4]:
ids = {}
for i in range(14):
    
    chro = 'chr'+str(i+1)
    path = 'C:\\Users\\ANTONIS\\Desktop\\test1\\'+chro+'train.lgen'
    
    if os.path.exists(path):
        f = open(path,'r')
    
        for line in f:
            patientsPhen[line.split()[0]].addSnps(line.split()[2], line.split()[3],line.split()[4])
    

    
for i in range(14):
    
    chro = 'chr'+str(i+1)
    
    for snp in chromosomes[chro].keys():
        
        allele1 = chromosomes[chro][snp][0]
        allele2 = chromosomes[chro][snp][1]
    
        for patient in patientsPhen.keys():
            
            patientsPhen[patient].snpCode(snp,allele1,allele2)
            
#patientsPhen = createDataSet('train',chromosomes,patientsPhen)

count = 0

for patient in patientsPhen.keys():
            
    if patientsPhen[patient].getSize() == len(minorSnps):
        
        count += 1
        
print("count is ",count)
print("snps is ",len(minorSnps))


ids['patients'] = setIdToName(list(patientsPhen.keys()))
ids['snps'] = setIdToName(minorSnps)
    
xTraining = np.zeros((len(ids['patients']['nameToId'].keys()), len(ids['snps']['nameToId'].keys())),dtype = int)

print("shape id ",xTraining.shape)
print("xtraining = ",len(xTraining))
print("xtraining.T = ",len(xTraining.T))
    
for i in range(len(xTraining)):
    for j in range(len(xTraining.T)):
        
        patient = ids['patients']['idToName'][i]
        snp = ids['snps']['idToName'][j]
        
        xTraining[i,j] = patientsPhen[patient].getSnpCode(snp)
        
        
yTraining = []

for i in range(len(ids['patients']['nameToId'].keys())):
    
    patient = ids['patients']['idToName'][i]
    yTraining.append(int(patientsPhen[patient].getCase()))

count is  4482
snps is  334
shape id  (4482, 334)
xtraining =  4482
xtraining.T =  334


In [5]:
#dok = SaveDataSet("dok",ids,xTraining)

# TEST SET CREATION

In [6]:
patientsPhenTest = {}

f = open('C:\\Users\\Antonis\\Documents\\GitHub\\diplwmatikh\\data\\phenotype_euro_test.txt','r')
f.readline()
for line in f:
    patientsPhenTest[line.split()[0]] = PatientPhenotype(line.split()[0],line.split()[3],line.split()[1],line.split()[2])  
    
    
path = 'C:\\Users\\ANTONIS\\Desktop\\test1\\'+'testPatient.txt'
write = open(path,'w')
for patient in patientsPhenTest.keys():
    write.write(patient + '\n')
            
write.close()

# run test_lgen bat

In [7]:
idsTest = {}

for i in range(14):
    
    chro = 'chr'+str(i+1)
    path = 'C:\\Users\\ANTONIS\\Desktop\\test1\\'+chro+'test.lgen'
    if os.path.exists(path):
        f = open(path,'r')
    
        for line in f:
            patientsPhenTest[line.split()[0]].addSnps(line.split()[2], line.split()[3],line.split()[4])
    

    
for i in range(14):
    
    chro = 'chr'+str(i+1)
    
    for snp in chromosomes[chro].keys():
        
        allele1 = chromosomes[chro][snp][0]
        allele2 = chromosomes[chro][snp][1]
        
        for patient in patientsPhenTest.keys():
            
            patientsPhenTest[patient].snpCode(snp,allele1,allele2)
            
#patientsPhenTest = createDataSet('test',chromosomes,patientsPhenTest)            

    
count = 0

for patient in patientsPhenTest.keys():
            
    if patientsPhenTest[patient].getSize() == len(minorSnps):
        
        count += 1
        
print("count is Test",count)
print("snps is Test",len(minorSnps))


idsTest['patients'] = setIdToName(list(patientsPhenTest.keys()))
idsTest['snps'] = setIdToName(minorSnps)
    
xTest = np.zeros((len(idsTest['patients']['nameToId'].keys()), len(idsTest['snps']['nameToId'].keys())),dtype = int)

print("shape id Test",xTest.shape)
print("xtraining = ",len(xTest))
print("xtest.T = ",len(xTest.T))
    
for i in range(len(xTest)):
    for j in range(len(xTest.T)):
        
        patient = idsTest['patients']['idToName'][i]
        snp = idsTest['snps']['idToName'][j]
        
        xTest[i,j] = patientsPhenTest[patient].getSnpCode(snp)
        
        
yTest = []

for i in range(len(idsTest['patients']['nameToId'].keys())):
    patient = idsTest['patients']['idToName'][i]
    yTest.append(int(patientsPhenTest[patient].getCase()))

yTest = np.array(yTest)

count is Test 498
snps is Test 334
shape id Test (498, 334)
xtraining =  498
xtest.T =  334


# Linear Regression

In [76]:
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

regr = linear_model.LinearRegression()
regr.fit(xTraining, yTraining)
yPredict1 = regr.predict(xTest)

for i in range(len(yPredict1)):
    if yPredict1[i] < 0.27:
        yPredict1[i] = 0
    else:
        yPredict1[i] = 1

error1 = mean_squared_error(yTest, yPredict1)
print("error 1 = ",error1)
print(metrics.accuracy_score(yTest,yPredict1))
print(metrics.confusion_matrix(yTest,yPredict1))

error 1 =  0.301204819277
0.698795180723
[[290 100]
 [ 50  58]]


# SVM

In [59]:
from sklearn.svm import SVC
clf = SVC()
clf.fit(xTraining, yTraining)
yPredict2 = clf.predict(xTest)
print(metrics.accuracy_score(yTest,yPredict2))
print(metrics.confusion_matrix(yTest,yPredict2))
error2 = mean_squared_error(yTest, yPredict2)
print("error 2 = ",error2)

0.78313253012
[[390   0]
 [108   0]]
error 2 =  0.21686746988


# RF

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn import grid_search

regr = linear_model.LinearRegression()
rfr = RandomForestRegressor(n_estimators = 500, random_state = 2016, verbose = 20)
param_grid = {}
model = grid_search.GridSearchCV(estimator = rfr, param_grid = param_grid, cv = 4, verbose = 20, scoring=RMSE)

model.fit(xTraining, yTraining)
yPredict3 = model.predict(xTest)
yPredict3 = my_forest.predict(xTest)

print(metrics.accuracy_score(yTest,yPredict3))
print(metrics.confusion_matrix(yTest,yPredict3))

error3 = mean_squared_error(yTest, yPredict3)
print("error 3 = ",error3)

Fitting 4 folds for each of 1 candidates, totalling 4 fits
[CV]  ................................................................
building tree 1 of 500


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


building tree 2 of 500


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.6s remaining:    0.0s


building tree 3 of 500
building tree 4 of 500


...

building tree 16 of 500
building tree 17 of 500


[Parallel(n_jobs=1)]: Done  17 out of  17 | elapsed:    4.3s remaining:    0.0s


building tree 18 of 500


[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    4.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    4.7s remaining:    0.0s


building tree 19 of 500
building tree 20 of 500
building tree 21 of 500
building tree 22 of 500
building tree 23 of 500
building tree 24 of 500
building tree 25 of 500
building tree 26 of 500
building tree 27 of 500
building tree 28 of 500
building tree 29 of 500
building tree 30 of 500
building tree 31 of 500
building tree 32 of 500
building tree 33 of 500
building tree 34 of 500
building tree 35 of 500
building tree 36 of 500
building tree 37 of 500
building tree 38 of 500
building tree 39 of 500
building tree 40 of 500
building tree 41 of 500
building tree 42 of 500
building tree 43 of 500
building tree 44 of 500
building tree 45 of 500
building tree 46 of 500
building tree 47 of 500
building tree 48 of 500
building tree 49 of 500
building tree 50 of 500
building tree 51 of 500
building tree 52 of 500
building tree 53 of 500
building tree 54 of 500
building tree 55 of 500
building tree 56 of 500
building tree 57 of 500
building tree 58 of 500
building tree 59 of 500
building tree 60

# LINEAR LOGISTIC REGRESSION 

In [ ]:
import sklearn.linear_model as linear_model
import sklearn.metrics as metrics
from sklearn import preprocessing
lr_clf = linear_model.LogisticRegression()
lr_clf.fit(xTraining, yTraining)
yPredict4 = lr_clf.predict(xTest)

print(metrics.accuracy_score(yTest,yPredict4))
print(metrics.confusion_matrix(yTest,yPredict4))
error4 = mean_squared_error(yTest, yPredict4)
print("error 4 = ",error4)

RMSE = mean_squared_error(yTest,yPredict4)**0.5
print("RMSE = ",RMSE)